In [1]:
import json
#from google.colab import files # solo para google collab
from google.cloud import storage
import pandas as pd
print(storage.__version__)

2.8.0


In [6]:
# google collab
# solo se ejecuta la primera vez: cargo el json con las credenciales del api key
#object_uploaded = files.upload()

# google collab
path_api_key = '/content/extreme-unison-399121-cadd77c555ca.json'

# entorno local
#path_api_key = 'gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

In [7]:
# carga de datos
# unicamente para .py en VM:
#data_business_reviews_norm = pd.read_parquet('gs://1_transform/data_business_reviews_norm.parquet')

# para entorno Google Colab o maquina local:

# cambiar segun bucket
bucket = client_storage.get_bucket('1_transform')

# cambiar segun dataframe
data_blob = bucket.blob('business_reviews_norm.parquet')
data_business_reviews = pd.read_parquet(storage.fileio.BlobReader(data_blob))
data_business_reviews.info()
data_business_reviews.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1531095 entries, 0 to 415753
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   business_id         1531095 non-null  object 
 1   user_id             1531095 non-null  object 
 2   rating              1531095 non-null  float64
 3   user_text           1101628 non-null  object 
 4   user_time_year      1531095 non-null  int32  
 5   user_time_month     1531095 non-null  int32  
 6   user_time_day       1531095 non-null  int32  
 7   user_time_hms       1531095 non-null  object 
 8   state_name          1531095 non-null  object 
 9   state_code          1531095 non-null  object 
 10  codigo_postal_zcta  1531095 non-null  int64  
dtypes: float64(1), int32(3), int64(1), object(6)
memory usage: 122.7+ MB


,business_id,user_id,rating,user_text,user_time_year,user_time_month,user_time_day,user_time_hms,state_name,state_code,codigo_postal_zcta
0,0x88892bb449d44cb3:0x7beaf0b2e3171a49,110771718750038252970,2.0,"No masks, requested curbside and waited 10 min...",2021,8,27,02:29:34.725000,alabama,AL,35115
1,0x88892bb449d44cb3:0x7beaf0b2e3171a49,104877410053821600291,5.0,This location does an awesome job! The delive...,2020,10,10,02:42:56.261000,alabama,AL,35115


In [10]:
columns_interes = [
    "state_name","state_code","codigo_postal_zcta",
    "business_id",
    "user_time_year","user_time_month","user_time_day",
    "rating"]

business_kpi_popularidad = data_business_reviews[
    columns_interes].sort_values(
        by=[
            "state_name","state_code","codigo_postal_zcta","business_id",
            "user_time_year","user_time_month","user_time_day"]
        ).copy() # type: ignore
business_kpi_popularidad.info()
business_kpi_popularidad.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1531095 entries, 279847 to 924453
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   state_name          1531095 non-null  object 
 1   state_code          1531095 non-null  object 
 2   codigo_postal_zcta  1531095 non-null  int64  
 3   business_id         1531095 non-null  object 
 4   user_time_year      1531095 non-null  int32  
 5   user_time_month     1531095 non-null  int32  
 6   user_time_day       1531095 non-null  int32  
 7   rating              1531095 non-null  float64
dtypes: float64(1), int32(3), int64(1), object(3)
memory usage: 87.6+ MB


,state_name,state_code,codigo_postal_zcta,business_id,user_time_year,user_time_month,user_time_day,rating
279847,alabama,AL,30165,0x888aa302e353cc37:0x82cdb8905e887821,2020,10,1,3.0
278486,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,1,4.0


In [11]:
business_kpi_popularidad_rating_business_id = business_kpi_popularidad.groupby(
    by=[
        "state_name","state_code","codigo_postal_zcta",
        "business_id","user_time_year","user_time_month"
        ],
    dropna=False
    )["rating"].size().reset_index().rename(columns={"rating":"business_id_rating_size"})
business_kpi_popularidad_rating_business_id["business_id_size_pctchange"] = business_kpi_popularidad_rating_business_id.groupby(
    by=[
        "state_name","state_code","codigo_postal_zcta",
        "business_id","user_time_year"
        ],
    dropna=False
    )["business_id_rating_size"].pct_change(1)*100
business_kpi_popularidad_rating_business_id.head(2)

,state_name,state_code,codigo_postal_zcta,business_id,user_time_year,user_time_month,business_id_rating_size,business_id_size_pctchange
0,alabama,AL,30165,0x888aa302e353cc37:0x82cdb8905e887821,2020,10,1,NaN
1,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,8,NaN


In [12]:
business_kpi_popularidad_rating_codigo_postal_zcta = business_kpi_popularidad.groupby(
     by=[
        "state_name","state_code","codigo_postal_zcta",
        "user_time_year","user_time_month"
        ],
    dropna=False
    )["rating"].size().reset_index().rename(columns={"rating":"codigo_postal_zcta_rating_size"})
business_kpi_popularidad_rating_codigo_postal_zcta["codigo_postal_zcta_size_pctchange"] = business_kpi_popularidad_rating_codigo_postal_zcta.groupby(
    by=[
        "state_name","state_code","codigo_postal_zcta",
        "user_time_year"
        ],
    dropna=False
    )["codigo_postal_zcta_rating_size"].pct_change(1)*100
business_kpi_popularidad_rating_codigo_postal_zcta.head()

,state_name,state_code,codigo_postal_zcta,user_time_year,user_time_month,codigo_postal_zcta_rating_size,codigo_postal_zcta_size_pctchange
0,alabama,AL,30165,2020,7,56,NaN
1,alabama,AL,30165,2020,8,42,-25.000000
2,alabama,AL,30165,2020,9,66,57.142857
3,alabama,AL,30165,2020,10,63,-4.545455
4,alabama,AL,30165,2020,11,56,-11.111111


In [14]:
business_kpi_popularidad_crecimiento = pd.merge(
    left=business_kpi_popularidad_rating_business_id.drop(columns=["business_id_rating_size"]),
    right=business_kpi_popularidad_rating_codigo_postal_zcta.drop(columns=["codigo_postal_zcta_rating_size"]),
    how='left'
    ).dropna()
business_kpi_popularidad_crecimiento["kpi_popularidad_crecimiento"] = business_kpi_popularidad_crecimiento[
    "business_id_size_pctchange"] - business_kpi_popularidad_crecimiento["codigo_postal_zcta_size_pctchange"]
business_kpi_popularidad_crecimiento.drop(
    columns=["business_id_size_pctchange","codigo_postal_zcta_size_pctchange"], inplace=True)
business_kpi_popularidad_crecimiento.info()
business_kpi_popularidad_crecimiento.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356809 entries, 2 to 527195
Data columns (total 7 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   state_name                   356809 non-null  object 
 1   state_code                   356809 non-null  object 
 2   codigo_postal_zcta           356809 non-null  int64  
 3   business_id                  356809 non-null  object 
 4   user_time_year               356809 non-null  int64  
 5   user_time_month              356809 non-null  int64  
 6   kpi_popularidad_crecimiento  356809 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 21.8+ MB


,state_name,state_code,codigo_postal_zcta,business_id,user_time_year,user_time_month,kpi_popularidad_crecimiento
2,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,8,-12.500000
3,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,9,-57.142857
4,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,10,64.545455
5,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,11,36.111111
6,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,12,3.928571


In [16]:
business_kpi_popularidad_parte = pd.merge(
    left=business_kpi_popularidad_rating_business_id.drop(columns=["business_id_size_pctchange"]),
    right=business_kpi_popularidad_rating_codigo_postal_zcta.drop(columns=["codigo_postal_zcta_size_pctchange"]),
    how='left'
    ).dropna()
business_kpi_popularidad_parte["kpi_popularidad_parte"] = (
    business_kpi_popularidad_parte["business_id_rating_size"] / business_kpi_popularidad_parte["codigo_postal_zcta_rating_size"]
    )*100
business_kpi_popularidad_parte.drop(
    columns=["business_id_rating_size","codigo_postal_zcta_rating_size"], inplace=True)
business_kpi_popularidad_parte.info()
business_kpi_popularidad_parte.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 527196 entries, 0 to 527195
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   state_name             527196 non-null  object 
 1   state_code             527196 non-null  object 
 2   codigo_postal_zcta     527196 non-null  int64  
 3   business_id            527196 non-null  object 
 4   user_time_year         527196 non-null  int64  
 5   user_time_month        527196 non-null  int64  
 6   kpi_popularidad_parte  527196 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 32.2+ MB


,state_name,state_code,codigo_postal_zcta,business_id,user_time_year,user_time_month,kpi_popularidad_parte
0,alabama,AL,30165,0x888aa302e353cc37:0x82cdb8905e887821,2020,10,1.587302
1,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,14.285714
2,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,8,11.904762
3,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,9,7.575758
4,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,10,12.698413


In [17]:
business_kpi_popularidad = pd.merge(
    left=business_kpi_popularidad_parte,
    right=business_kpi_popularidad_crecimiento,
    how='left')
business_kpi_popularidad.info()
business_kpi_popularidad.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 527196 entries, 0 to 527195
Data columns (total 8 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   state_name                   527196 non-null  object 
 1   state_code                   527196 non-null  object 
 2   codigo_postal_zcta           527196 non-null  int64  
 3   business_id                  527196 non-null  object 
 4   user_time_year               527196 non-null  int64  
 5   user_time_month              527196 non-null  int64  
 6   kpi_popularidad_parte        527196 non-null  float64
 7   kpi_popularidad_crecimiento  356809 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 36.2+ MB


,state_name,state_code,codigo_postal_zcta,business_id,user_time_year,user_time_month,kpi_popularidad_parte,kpi_popularidad_crecimiento
0,alabama,AL,30165,0x888aa302e353cc37:0x82cdb8905e887821,2020,10,1.587302,NaN
1,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,7,14.285714,NaN
2,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,8,11.904762,-12.500000
3,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,9,7.575758,-57.142857
4,alabama,AL,30165,0x888aa3124d2f18fd:0xe6945c7d144c4402,2020,10,12.698413,64.545455
